In [1]:
from dotenv import load_dotenv 
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/home/lawal/miniconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rage", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeConfigurationError: You haven't specified an Api-Key.

In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
processed_data = []
client = OpenAI()